# Auto Big-bench
copyright 2024, Denis Rothman

*The potential of next generation of AI is to be able to evaluate and benchmark itself for a large range of tasks.*

**June 24, 2024 upgrade** improved processing speed measured with time metric

**May 15, 2024 upgrade** from `gpt-4` to `gpt4-o`

[Big-bench](https://github.com/google/BIG-bench/blob/main/bigbench/benchmark_tasks/README.md) contains more than 200+ NLP tasks. The goal is to evaluate a model.

In this notebook, we take ChatGPT-GPT-4o a step further. We will not ask ChatGPT-GPT-4o to solve a Big-bench NLP problem and apply metrics. We will ask GPT-4o to create the tasks itself and solve them!

The program will feed GPT-4o a sample of 140+ Big-bench tasks with a two-part prompt:   

**The first part contains the instruction:**    

**The second part is the description of a Big-bench:**  

**The output will then be displayed for human evaluation**
Human evaluation plays an important role in LLM training and evaluations. Reinforcement Learning with Human Feedback(RLHF) will help mitigate the potential limits of automated models and evaluation metrics.

**Limit of the program:** The program does not run thousands of samples for each task. The goal is to show the potential of Large Language Models(LLMs)

**Potential:** We can see that GPT-4o, PaLM 2, and other Foundations Models are just the beginning of what will become *Massive Multitask Language Understanding(MMLU)* models in one form or another in the years to come.

# Install OpenAI

**June 24 update** OpenAI has updated its installation. There is no need to install cohere and tiktoken separately. OpenAI fixed the issue!
This installation has been now shortened to strictly OpenAI.

Note:
OpenAI in January 2024 requires dependencies that in turn require other dependencies that are installed after or before. On Google Colab(or your machine), if you encounter OpenAI installation issues, try the process below:  
a) uncomment and install cohere and run it after the `!pip install openai` cell<br>
b) uncomment and install tiktoken and run it after `!pip install cohere `  
c) then run `!pip install cohere ` again   
d) then run `!pip install openai` again   

Hopefully, OpenAI will fix this in 2024 and we will update the installation accordingly. This is normal on a fast-moving market. We simply need to be on the watch.

In [1]:
#Importing openai
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [4]:
#API Key
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/api_key.txt", "r")
API_KEY=f.readline()
f.close()

Mounted at /content/drive


In [5]:
#The OpenAI Key
import openai
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

# Retrieve the list of Big-bench prompts designed for this notebook

The list was created from the list of tasks of [Big-bench](https://github.com/google/BIG-bench/blob/main/bigbench/benchmark_tasks/README.md)

In [6]:
!curl -L https://raw.githubusercontent.com/Denis2054/Transformers_3rd_Edition/master/Chapter15/tasks.txt --output "tasks.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17838  100 17838    0     0  69259      0 --:--:-- --:--:-- --:--:-- 69408


# Read the file into a Pandas Dataframe

In [7]:
import pandas as pd

# read the file
df = pd.read_csv('tasks.txt', header=None, on_bad_lines='skip')

# If you want to add a column name after loading
df.columns = ['Tasks']

# print the dataframe
df

,Tasks
0,1.Explain the following task 2.Provide an exam...
1,1.Explain the following task 2.Provide an exam...
2,1.Explain the following task 2.Provide an exam...
3,1.Explain the following task 2.Provide an exam...
4,1.Explain the following task 2.Provide an exam...
...,...
139,1.Explain the following task 2.Provide an exam...
140,1.Explain the following task 2.Provide an exam...
141,1.Explain the following task 2.Provide an exam...
142,1.Explain the following task 2.Provide an exam...


In [8]:
nbt=len(df)
print("Number of tasks: ", nbt)

Number of tasks:  144


# Defining the rolel of the model

In [9]:
import openai
from openai import OpenAI

client = OpenAI()
gptmodel="gpt-4o" # or select gpt-3.5-turbo

def openai_chat(input_text):
    response = client.chat.completions.create(
        model=gptmodel,
        messages=[
            {"role": "system", "content": "You are an expert Natural Language Processing exercise expert."},
            {"role": "assistant", "content": "1.You can explain any NLP task. 2.Create an example 3.Solve the example"},
            {"role": "user", "content": input_text}
        ],
        temperature=0.1  # Add the temperature parameter here and other parameters you need
    )
    return response.choices[0].message.content

# Displaying the response of the model

In [10]:
from IPython.core.display import display, HTML
def display_response(input_text, response, bb_task):
  html_content = f"""
  <!DOCTYPE html>
  <html>
  <head>
      <title>Big-bench Tasks</title>
      <style>
        p {{
            max-width: 600px;
        }}
    </style>
  </head>
  <body>
      <h1>{bb_task}</h1>
      <p>{task}</p>
  </body>
  </html>
  """

  # And finally we display it
  display(HTML(html_content))
  html_file = open("output.html", "a")
  html_file.write(html_content)
  html_file.close()

In [11]:
html_file = open("output.html", "w") #just to make sure a new file is created before running the tasks to avoid
html_file.close()                    #processing large files.

# Running the tasks

Check OpenAI's policy for rate limits before running the tasks:
https://platform.openai.com/docs/guides/rate-limits/overview


In [12]:
import time

start_time = time.time()  # Start timing before the request

counter = 0
nb_tasks = nbt
for index, row in df.iterrows():
    input_text = row['Tasks']                 # the complete prompt
    counter += 1                              # task counter
    if counter > nb_tasks:
        break                                 # nb of tasks
    task = openai_chat(input_text)            # model call
    task = task.replace('\n', '<br>')         # formatting the output
    parts = input_text.split('Solve it:')     # extracting the task from the input
    bb_task = parts[1].strip()                # The strip() function
    display_response(input_text, task, bb_task) # displaying the task and response

    if counter % 20 == 0:                     # if the counter is divisible by 50
        print(f"Processed {counter} tasks.")
        #time.sleep(60)                        # pause for 60 seconds

response_time = time.time() - start_time  # Measure response time
print(f"Response Time: {response_time:.2f} seconds")  # Print response time

Processed 20 tasks.


Processed 40 tasks.


Processed 60 tasks.


Processed 80 tasks.


Processed 100 tasks.


Processed 120 tasks.


Processed 140 tasks.


Response Time: 1659.37 seconds
